In [1]:
import os

import pandas as pd
import numpy as np
import sqlite3
import pandas 

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

import seaborn as sns

# Import SQLAlchemy `automap` and other dependencies here
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

app = Flask(__name__)

In [2]:
engine = create_engine("sqlite:///meter_restaurant.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [3]:
meters = Base.classes.meters
restaurant_yelp = Base.classes.restaurant_yelp
meter_restaurant = Base.classes.meter_restaurant

In [4]:
connect = sqlite3.connect('meter_restaurant.sqlite')
df = pd.read_sql_query("SELECT * FROM restaurant_yelp", connect)
df.head()

,yelp_id,alias,category_alias,category_title,latitude,longitude,display_phone,distance,image_url,is_closed,...,state,zip_code,name,phone,price,rating,review_count,url,total_likes,likes
0,wGl_DyNxSv8KUtYgiuLhmA,bi-rite-creamery-san-francisco,icecream,Desserts,37.761591,-122.425717,(415) 626-5600,946.386738,https://s3-media2.fl.yelpcdn.com/bphoto/tLwFRw...,FALSE,...,CA,94110,Bi-Rite Creamery,1.415627e+10,$,4.5,9467,https://www.yelp.com/biz/bi-rite-creamery-san-...,None,12
1,lJAGnYzku5zSaLnQ_T6_GQ,brendas-french-soul-food-san-francisco,breakfast_brunch,Breakfast & Brunch,37.782902,-122.419043,(415) 345-8100,2885.389131,https://s3-media3.fl.yelpcdn.com/bphoto/mEzEYZ...,FALSE,...,CA,94102,Brenda's French Soul Food,1.415346e+10,$$,4.0,10505,https://www.yelp.com/biz/brendas-french-soul-f...,None,5
2,WavvLdfdP6g8aZTtbBQHTw,gary-danko-san-francisco,newamerican,American,37.805870,-122.420580,(415) 749-2060,5191.341803,https://s3-media3.fl.yelpcdn.com/bphoto/CAprIo...,FALSE,...,CA,94109,Gary Danko,1.415749e+10,$$$$,4.5,5389,https://www.yelp.com/biz/gary-danko-san-franci...,None,6
3,ri7UUYmx21AgSpRsf4-9QA,tartine-bakery-and-cafe-san-francisco,bakeries,Bakeries,37.761425,-122.424051,(415) 487-2600,1091.249586,https://s3-media2.fl.yelpcdn.com/bphoto/6XiycX...,FALSE,...,CA,94110,Tartine Bakery & Cafe,1.415487e+10,$$,4.0,7640,https://www.yelp.com/biz/tartine-bakery-and-ca...,None,1
4,76smcUUGRvq3k1MVPUXbnA,mitchells-ice-cream-san-francisco,icecream,Desserts,37.744231,-122.422816,(415) 648-2300,2207.086160,https://s3-media4.fl.yelpcdn.com/bphoto/-BBzTT...,FALSE,...,CA,94110,Mitchell's Ice Cream,1.415648e+10,$,4.5,3846,https://www.yelp.com/biz/mitchells-ice-cream-s...,None,1


In [5]:
list(df.columns) 

['yelp_id',
 'alias',
 'category_alias',
 'category_title',
 'latitude',
 'longitude',
 'display_phone',
 'distance',
 'image_url',
 'is_closed',
 'address',
 'address2',
 'address3',
 'city',
 'country',
 'state',
 'zip_code',
 'name',
 'phone',
 'price',
 'rating',
 'review_count',
 'url',
 'total_likes',
 'likes']

In [6]:
inspector = inspect(engine)
inspector.get_table_names()


['meter_restaurant', 'meters', 'restaurant_yelp']

In [7]:
# columns = inspector.get_columns('restaurant_yelp')
# for column in columns:
#     print(column['name'], column["type"])
# rating = df.groupby('rating').size().sort_values(ascending=False)
# rating
# price = df.groupby('price').size().sort_values(ascending=False)
# price

In [8]:
session = Session(engine)

In [9]:
# What are the most popular cuisine?
# List the cuisine and the reviews in descending order.
food_type = session.query(restaurant_yelp.category_title, func.count(restaurant_yelp.review_count)).\
    group_by(restaurant_yelp.category_title).order_by(func.count(restaurant_yelp.review_count).desc()).all()
# food_type

In [10]:
sel = [restaurant_yelp.category_title, 
       func.sum(restaurant_yelp.review_count),
       func.avg(restaurant_yelp.review_count)]

results = session.query(*sel).group_by(restaurant_yelp.category_title).order_by(restaurant_yelp.category_title).all()
# results

In [11]:
df3 = pd.read_sql_query("SELECT * FROM meter_restaurant", connect)
df3.head()

,yelp_id,meter1,meter2,meter3,meter4,meter5,meter6,meter7,meter8,meter9,meter10,resturant_name
0,wGl_DyNxSv8KUtYgiuLhmA,223043,223831,223429,223027,223217,223068,223466,223693,223996,223476,Bi-Rite Creamery
1,lJAGnYzku5zSaLnQ_T6_GQ,213043,213075,217056,217544,217102,213013,217307,213684,213018,213410,Brenda's French Soul Food
2,WavvLdfdP6g8aZTtbBQHTw,206810,206497,206813,206665,206605,206801,206403,207007,206530,206144,Gary Danko
3,ri7UUYmx21AgSpRsf4-9QA,223772,223476,223361,223693,223987,223831,223734,223068,223429,223027,Tartine Bakery & Cafe
4,76smcUUGRvq3k1MVPUXbnA,221741,221993,221915,221745,221811,221740,221095,221661,221800,221876,Mitchell's Ice Cream


In [12]:
df2 = pd.read_sql_query("SELECT * FROM meters", connect)
df2.head()

,objectid,street_num,street_name,longitude,latitude,meter_type
0,197012,436,CALIFORNIA ST,-122.402335,37.792967,SS
1,197013,109,JACKSON ST,-122.398770,37.796977,SS
2,197014,432,CALIFORNIA ST,-122.402191,37.792986,SS
3,197015,221,SANSOME ST,-122.401150,37.792757,SS
4,197016,133,STEUART ST,-122.392658,37.792929,SS


In [14]:
meter_df = pd.read_sql_query("SELECT * FROM meters", connect)
meter_df


,objectid,street_num,street_name,longitude,latitude,meter_type
0,197012,436,CALIFORNIA ST,-122.402335,37.792967,SS
1,197013,109,JACKSON ST,-122.398770,37.796977,SS
2,197014,432,CALIFORNIA ST,-122.402191,37.792986,SS
3,197015,221,SANSOME ST,-122.401150,37.792757,SS
4,197016,133,STEUART ST,-122.392658,37.792929,SS
5,197017,27,EMBARCADERO NORTH,-122.394854,37.795619,SS
6,197018,331,EMBARCADERO NORTH,-122.396884,37.798065,SS
7,197019,208,WASHINGTON ST,-122.398594,37.796179,MS
8,197020,3,DRUMM ST,-122.396549,37.793841,SS
9,197021,209,FRONT ST,-122.398943,37.793746,SS


In [24]:
lat=meter_df['latitude']
long=meter_df['longitude']
metersid=meter_df['objectid']

url = 'http://api.sfpark.org/sfpark/rest/availabilityservice?'
query_url=f"{url}lat={lat}&long={long}&radius=0.01&uom=mile&response=json"

meterstatus = []

In [ ]:
import requests

i=1
for meter in metersid:
    try:
            response = requests.get(query_url).json()
            if response['cod'] == 200:
                print("Processing Record " + str(i) + " of Set 1 | " + meter)
                #city_data.append(response)
                meterstatus.append(response['AVL']['TYPE'])
                
            else:
                print(f"meter not found,skipping...")
    except:
            print(f" {resonse} Exception...")

    i=i+1


 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Excep

 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Excep

 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Excep

 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Excep

 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Exception...
 Excep

In [ ]:
import random
meter_status=[]
for meter in metersid:
    my_randoms.append(random.randrange(0,1)
    print (meter_status)